In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
sns.set(style="whitegrid")
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import accuracy_score, roc_auc_score
import reverse_geocoder as rg
import gmplot
import pickle
from matplotlib import pyplot
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GroupKFold
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pylab import rcParams
rcParams['figure.figsize'] = 9,10

In [4]:
train=pd.read_csv('train_set.csv')
test=pd.read_csv('test_set.csv')

In [68]:
def prnt(u_id, how='all', tbl=train_agg, mcc=None, field=None ):
    if mcc : cond=(train['mcc']==mcc)
    elif field: cond=(train[field]==1)
    else: cond=True
    tmp=tbl[(tbl.customer_id==u_id)&(cond)][['b_lat' ,'b_lon']].dropna().drop_duplicates()
    gmap = gmplot.GoogleMapPlotter(55.874, 37.445, 2)
    gmap.scatter(tmp.b_lat, tmp.b_lon, 'cornflowerblue', marker=True)
    if 'work_add_lat' in tbl.columns:
        if tbl[tbl.customer_id==u_id]['work_add_lat'].any()&tbl[tbl.customer_id==u_id]['work_add_lon'].any():
            gmap.scatter([tbl[tbl.customer_id==u_id]['work_add_lat'].iloc[0]], [ tbl[tbl.customer_id==u_id]['work_add_lon'].iloc[0] ], 'y', marker=True)
        if tbl[tbl.customer_id==u_id]['home_add_lat'].any()&tbl[tbl.customer_id==u_id]['home_add_lon'].any():
            gmap.scatter([tbl[tbl.customer_id==u_id]['home_add_lat'].iloc[0]], [tbl[tbl.customer_id==u_id]['home_add_lon'].iloc[0]], 'k', marker=True)

    gmap.draw("mymap.html")
    pass

# Чистим данные

In [5]:
#Перед тем как удалить заграничные транзакии, посчитаем сколько их было. 
train.loc[train[train.country=='RU '].index,'country']='RUS'
test.loc[test[test.country=='RU '].index,'country']='RUS'
train['country_nm']=train.groupby('customer_id')['country'].transform(lambda x: len(x.unique()))
test['country_nm']=test.groupby('customer_id')['country'].transform(lambda x: len(x.unique()))
train['country_am']=train.groupby('customer_id')['country'].transform(lambda x: x[x!='RUS'].count())
test['country_am']=test.groupby('customer_id')['country'].transform(lambda x: x[x!='RUS'].count())

In [6]:
train.drop(train[ train.country!='RUS'].index, axis=0, inplace=True)
test.drop(test[ test.country!='RUS'].index, axis=0, inplace=True)

# Даты

In [7]:
#обработаем даты
train['transaction_date']=pd.to_datetime(train['transaction_date'], format='%Y-%m-%d')
test['transaction_date']=pd.to_datetime(test['transaction_date'], format='%Y-%m-%d')

In [8]:
#добавим признак день недели
train['day_of_week']=train['transaction_date'].apply(lambda x: x.dayofweek)
test['day_of_week']=test['transaction_date'].apply(lambda x: x.dayofweek)

In [9]:
#выходной
train['day_off']=train['day_of_week'].apply(lambda x: 1 if x in[5,6] else 0)
test['day_off']=test['day_of_week'].apply(lambda x: 1 if x in[5,6] else 0)
clb=['2017-02-23','2017-02-24','2017-03-08',
                                   '2017-05-01','2017-05-08','2017-05-09',
                                  '2017-06-12','2017-11-06']
train.loc[train[train.transaction_date.isin(clb)].index,'day_off']=1
test.loc[test[test.transaction_date.isin(clb)].index,'day_off']=1

# Организационное 

In [10]:
#LabelEncoder на 'customer_id' и 'terminal_id'
le_c=LabelEncoder()
le_c.fit(train.customer_id.append(test.customer_id))
train['customer_id']=le_c.transform(train.customer_id)
test['customer_id']=le_c.transform(test.customer_id)

le_t=LabelEncoder()
le_t.fit(train.terminal_id.astype(str).append(test.terminal_id.astype(str)))
train['terminal_id']=le_t.transform(train.terminal_id.astype(str))
test['terminal_id']=le_t.transform(test.terminal_id.astype(str))

with open('le_c1.pkl', 'wb') as pkl:
    pickle.dump(le_c, pkl, protocol=2)
with open('le_t1.pkl', 'wb') as pkl:
    pickle.dump(le_c, pkl, protocol=2)

In [11]:
#сделаем данные чуть удобней, схлопнем поля для адресов и координат банкоматов.

train['is_atm']=(~train[['atm_address_lat', 'atm_address_lon']].isnull().any(axis=1)).astype(int)
temp=train.loc[:,['atm_address','pos_address','atm_address_lat', 'atm_address_lon', 'pos_adress_lat', 'pos_adress_lon']]
temp.fillna(value=pd.Series(data=['', '', 0,0,0,0], index=temp.columns.values), inplace=True)
temp['b_lat']=temp['atm_address_lat'].values+temp['pos_adress_lat'].values
temp['b_lon']=temp['atm_address_lon']+temp['pos_adress_lon']
temp['b_adr']=temp['atm_address']+temp['pos_address']
train[['b_lat', 'b_lon']]=temp[['b_lat', 'b_lon']]
train['b_adr']=temp['b_adr']

train.drop(['atm_address' ,'atm_address_lat' ,'atm_address_lon', 
            'pos_address' ,'pos_adress_lat','pos_adress_lon'], axis=1, inplace=True)


test['is_atm']=(~test[['atm_address_lat', 'atm_address_lon']].isnull().any(axis=1)).astype(int)
temp=test.loc[:,['atm_address','pos_address','atm_address_lat', 'atm_address_lon', 'pos_address_lat', 'pos_address_lon']]
temp.fillna(value=pd.Series(data=['', '', 0,0,0,0], index=temp.columns.values), inplace=True)
temp['b_lat']=temp['atm_address_lat'].values+temp['pos_address_lat'].values
temp['b_lon']=temp['atm_address_lon']+temp['pos_address_lon']
temp['b_adr']=temp['atm_address']+temp['pos_address']
test[['b_lat', 'b_lon']]=temp[['b_lat', 'b_lon']]
test['b_adr']=temp['b_adr']

test.drop(['atm_address' ,'atm_address_lat' ,'atm_address_lon', 
            'pos_address' ,'pos_address_lat','pos_address_lon'], axis=1, inplace=True)




In [12]:
#Усредним координаты банкоматов
atm_j=train[train.is_atm==1][['terminal_id','b_lat','b_lon']].append(
    test[test.is_atm==1][['terminal_id','b_lat','b_lon']], ignore_index=True)
mp_lat=atm_j.groupby('terminal_id')['b_lat'].mean()
mp_lon=atm_j.groupby('terminal_id')['b_lon'].mean()
train.loc[train[train.is_atm==1].index, 'b_lat']=train.terminal_id.map(mp_lat)
train.loc[train[train.is_atm==1].index, 'b_lon']=train.terminal_id.map(mp_lon)
test.loc[test[test.is_atm==1].index, 'b_lat']=test.terminal_id.map(mp_lat)
test.loc[test[test.is_atm==1].index, 'b_lon']=test.terminal_id.map(mp_lon)

In [13]:
#Добавим "целевую переменную"

def is_clth(x,r=0.02):
    if ((x[0]-x[2])**2+(x[1]-x[3])**2)**0.5<=r: ans=1
    else: ans=0
    return ans

train['is_home']=train[['home_add_lat' ,'home_add_lon', 'b_lat','b_lon']].apply(lambda x: is_clth(x), axis=1).astype(np.int16)
train['is_work']=train[['work_add_lat' ,'work_add_lon', 'b_lat','b_lon']].apply(lambda x: is_clth(x), axis=1).astype(np.int16)

In [14]:
#Удалим записи, с координатами банкоматов ==0

train.drop(train[(train.b_lat==0)|(train.b_lon==0)].index, axis=0,inplace=True)
test.drop(test[(test.b_lat==0)|(test.b_lon==0)].index, axis=0,inplace=True)


# mcc

In [69]:
#test mcc
test['mcc']=test.mcc.astype(str).apply(lambda x: x.replace(',', '')).fillna(0).astype(np.int16)
test.loc[157168,'mcc']=3400

In [71]:
#добавим избранные mcc которые кажутся важными

train['mcc5912']=train.mcc.apply(lambda x: 1 if x==5912 else 0) #аптеки
test['mcc5912']=test.mcc.apply(lambda x: 1 if x==5912 else 0) 

shops=[5921, 5311, 5331,5411,5412,5422,5451, 5499,5411,5462] # Категория продуктовые магазины
eat=[5812, 5813, 5814] #категория рестораны
fl=[5193, 5992] #цветы
train['shops']=train.mcc.apply(lambda x: 1 if x in shops else 0) .astype(np.int16)
test['shops']=test.mcc.apply(lambda x: 1 if x in shops else 0) .astype(np.int16)
train['eat']=train.mcc.apply(lambda x: 1 if x in eat else 0) .astype(np.int16)
test['eat']=test.mcc.apply(lambda x: 1 if x in eat else 0) .astype(np.int16)
train['fl']=train.mcc.apply(lambda x: 1 if x in fl else 0) .astype(np.int16)
test['fl']=test.mcc.apply(lambda x: 1 if x in fl else 0) .astype(np.int16)
train['outh']=((train['mcc5912']+train['shops']+train['eat']+train['fl'])==0).astype(np.int16) # все остальные
test['outh']=((test['mcc5912']+test['shops']+test['eat']+test['fl'])==0).astype(np.int16)

# Группировка внутри юзера

In [72]:
# сколько раз юзер был в этой точке, сколько всего для него было разных точек


def dist(x1,x2):
    d=((x1[0]-x2[0])**2+(x1[1]-x2[1])**2)**0.5
    return d


for user, sub_df in train.groupby('customer_id'): 
    #частота
    train.loc[sub_df.index,'fr']=sub_df.groupby(['b_lat','b_lon'])['city'].transform('count')
    
train['n_pnts']=train.groupby('customer_id')['city'].transform('count')
train['fr_ratio']=train.fr/train.n_pnts

#test


for user, sub_df in test.groupby('customer_id'): 
    #частота
    test.loc[sub_df.index,'fr']=sub_df.groupby(['b_lat','b_lon'])['city'].transform('count').astype(np.int16)
    
test['n_pnts']=test.groupby('customer_id')['city'].transform('count').astype(np.int16)
test['fr_ratio']=(test.fr/test.n_pnts).astype(np.float16)

In [73]:
#аггрегация по выходным. доля сколько в этой точке пользователь был в выходные и в будни
train['fr_w']=(train.groupby(['customer_id','b_lat', 'b_lon'])['day_off'].transform('sum')/train['fr']).astype(np.float16)
train['fr_b']=(train.groupby(['customer_id','b_lat', 'b_lon'])['day_off'].transform(lambda x: 
                                                                                np.sum(x==0))/train['fr']).astype(np.float16)
#аггрегация по выходным. доля сколько в этой точке пользователь был в выходные и в будни
test['fr_w']=(test.groupby(['customer_id','b_lat', 'b_lon'])['day_off'].transform('sum')/test['fr']).astype(np.float16)
test['fr_b']=(test.groupby(['customer_id','b_lat', 'b_lon'])['day_off'].transform(lambda x: 
                                                                                np.sum(x==0))/test['fr']).astype(np.float16)

In [77]:
#Перейдем к почти уникальным точкам (оставим деление на выходные):
train['amount_mean']=train.groupby(['customer_id','b_lat', 'b_lon'])['amount'].transform('mean').astype(np.float16)
test['amount_mean']=test.groupby(['customer_id','b_lat', 'b_lon'])['amount'].transform('mean').astype(np.float16)

train=train[['amount_mean','day_off','city','customer_id', 'home_add_lat',
       'home_add_lon','work_add_lat', 'work_add_lon', 'mcc', 'terminal_id', 'is_atm', 'b_lat', 'b_lon','is_home', 
             'is_work', 'mcc5912', 'shops', 'eat', 'fl', 'outh', 'fr',
       'dist_cm', 'dist_cm_med', 'n_pnts', 'fr_ratio', 'fr_w', 'fr_b']].drop_duplicates()

test=test[['amount_mean','day_off','city','customer_id',  'mcc', 'terminal_id', 'is_atm', 'b_lat', 'b_lon', 
           'mcc5912', 'shops', 'eat', 'fl', 'outh', 'fr',
       'dist_cm', 'dist_cm_med', 'n_pnts', 'fr_ratio', 'fr_w', 'fr_b']].drop_duplicates()


KeyError: 'Column not found: amount'

# Рассчеты внутри юзера с использованием NearestNeighbors

In [79]:
#функция с одним условием
def un_smth_1(X, cond, name):
    for user, sub_df in X.groupby('customer_id'): 
        neibh = NearestNeighbors(radius=0.02)
        tmp=sub_df[sub_df[cond]==1][['b_lat', 'b_lon']].drop_duplicates()
        if tmp.shape[0]==0: 
            X.loc[sub_df.index,name]=0
            continue
        neibh.fit(tmp[['b_lat', 'b_lon']].values)
        idx=neibh.radius_neighbors(sub_df[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
        X.loc[sub_df.index,name]=[len(x) for x in idx]
    X[name]=X[name].astype(np.int16)
    pass

#Функция с двумя условиями 
def un_smth_2(X, cond1, cond2, val2, name):
    for user, sub_df in X.groupby('customer_id'): 
        neibh = NearestNeighbors(radius=0.02)
        tmp=sub_df[(sub_df[cond1]==1)&(sub_df[cond2]==val2)][['b_lat', 'b_lon']].drop_duplicates()
        if tmp.shape[0]==0: 
            X.loc[sub_df.index,name]=0
            continue
        neibh.fit(tmp[['b_lat', 'b_lon']].values)
        idx=neibh.radius_neighbors(sub_df[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
        X.loc[sub_df.index,name]=[len(x) for x in idx]
    X[name]=X[name].astype(np.int16)
    pass

In [80]:
%%time
#уникальные соседи
for user, sub_df in train.groupby('customer_id'): 
    neibh = NearestNeighbors(radius=0.02)
    tmp=sub_df[['b_lat', 'b_lon']].drop_duplicates()
    neibh.fit(tmp[['b_lat', 'b_lon']].values)
    idx=neibh.radius_neighbors(sub_df[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
    train.loc[sub_df.index,'un_nb']=[len(x) for x in idx]

for user, sub_df in test.groupby('customer_id'): 
    neibh = NearestNeighbors(radius=0.02)
    tmp=sub_df[['b_lat', 'b_lon']].drop_duplicates()
    neibh.fit(tmp[['b_lat', 'b_lon']].values)
    idx=neibh.radius_neighbors(sub_df[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
    test.loc[sub_df.index,'un_nb']=[len(x) for x in idx]
train['un_nb']=train['un_nb'].astype(np.int16)
test['un_nb']=test['un_nb'].astype(np.int16)

CPU times: user 3min 43s, sys: 192 ms, total: 3min 43s
Wall time: 3min 43s


In [81]:
for user, sub_df in test.groupby('customer_id'): 
    neibh = NearestNeighbors(radius=0.02)
    tmp=sub_df[['b_lat', 'b_lon']].drop_duplicates()
    neibh.fit(tmp[['b_lat', 'b_lon']].values)
    idx=neibh.radius_neighbors(sub_df[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
    test.loc[sub_df.index,'un_nb']=[len(x) for x in idx]
train['un_nb']=train['un_nb'].astype(np.int16)
test['un_nb']=test['un_nb'].astype(np.int16)

In [82]:
%%time
for (i,j) in [('shops','un_nb_shops'), ('is_atm', 'un_nb_atm'), ('mcc5912','un_nb_5912' ), 
          ('eat','un_nb_eat') , ('fl','un_nb_fl'),('outh','un_nb_outh')]:
    un_smth_1(train, i, j)
    un_smth_1(test, i, j)
    print (1)

1
1
1
1
1
1
CPU times: user 22min 42s, sys: 896 ms, total: 22min 43s
Wall time: 22min 45s


In [84]:
%%time
for (i,j) in [('shops','un_nb_shops'), ('is_atm', 'un_nb_atm'), ('mcc5912','un_nb_5912' ), 
          ('eat','un_nb_eat') , ('fl','un_nb_fl'),('outh','un_nb_outh')]:
    for (k,m) in [(1,'_w'), (0,'_b')]:
        un_smth_2(train, i,'day_off',k, j+m)
        un_smth_2(test, i,'day_off',k, j+m)
        print (1)

1
1
1
1
1
1
1
1
1
1
1
1
CPU times: user 48min, sys: 2.13 s, total: 48min 2s
Wall time: 48min 5s


In [99]:
#где-то относительные значения лучше абсолютных
train['un_nb_shops_ratio']=(train.un_nb_shops/train.un_nb).fillna(0).astype(np.float16)
train['un_nb_5912_ratio']=(train.un_nb_5912/train.un_nb).fillna(0).astype(np.float16)
train['un_nb_eat_ratio']=(train.un_nb_eat/train.un_nb).fillna(0).astype(np.float16)
train['un_nb_atm_ratio']=(train.un_nb_atm/train.un_nb).fillna(0).astype(np.float16)
train['un_nb_fl_ratio']=(train.un_nb_fl/train.un_nb).fillna(0).astype(np.float16)
train['un_nb_atm_w_ratio']=(train.un_nb_atm_w/train.un_nb_atm).fillna(0).astype(np.float16)
train['un_nb_atm_b_ratio']=(train.un_nb_atm_b/train.un_nb_atm).fillna(0).astype(np.float16)
train['un_nb_shops_w_ratio']=(train.un_nb_shops_w/train.un_nb_shops).fillna(0).astype(np.float16)
train['un_nb_shops_b_ratio']=(train.un_nb_shops_b/train.un_nb_shops).fillna(0).astype(np.float16)
train['un_nb_5912_w_ratio']=(train.un_nb_5912_w/train.un_nb_5912).fillna(0).astype(np.float16)
train['un_nb_5912_b_ratio']=(train.un_nb_5912_b/train.un_nb_5912).fillna(0).astype(np.float16)
train['un_nb_eat_w_ratio']=(train.un_nb_eat_w/train.un_nb_eat).fillna(0).astype(np.float16)
train['un_nb_eat_b_ratio']=(train.un_nb_eat_b/train.un_nb_eat).fillna(0).astype(np.float16)
train['un_nb_fl_w_ratio']=(train.un_nb_fl_w/train.un_nb_fl).fillna(0).astype(np.float16)
train['un_nb_fl_b_ratio']=(train.un_nb_fl_b/train.un_nb_fl).fillna(0).astype(np.float16)
train['un_nb_outh_w_ratio']=(train.un_nb_outh_w/train.un_nb_outh).fillna(0).astype(np.float16)
train['un_nb_outh_b_ratio']=(train.un_nb_outh_b/train.un_nb_outh).fillna(0).astype(np.float16)

In [100]:
test['un_nb_shops_ratio']=(test.un_nb_shops/test.un_nb).fillna(0).astype(np.float16)
test['un_nb_5912_ratio']=(test.un_nb_5912/test.un_nb).fillna(0).astype(np.float16)
test['un_nb_eat_ratio']=(test.un_nb_eat/test.un_nb).fillna(0).astype(np.float16)
test['un_nb_atm_ratio']=(test.un_nb_atm/test.un_nb).fillna(0).astype(np.float16)
test['un_nb_fl_ratio']=(test.un_nb_fl/test.un_nb).fillna(0).astype(np.float16)
test['un_nb_atm_w_ratio']=(test.un_nb_atm_w/test.un_nb_atm).fillna(0).astype(np.float16)
test['un_nb_atm_b_ratio']=(test.un_nb_atm_b/test.un_nb_atm).fillna(0).astype(np.float16)
test['un_nb_shops_w_ratio']=(test.un_nb_shops_w/test.un_nb_shops).fillna(0).astype(np.float16)
test['un_nb_shops_b_ratio']=(test.un_nb_shops_b/test.un_nb_shops).fillna(0).astype(np.float16)
test['un_nb_5912_w_ratio']=(test.un_nb_5912_w/test.un_nb_5912).fillna(0).astype(np.float16)
test['un_nb_5912_b_ratio']=(test.un_nb_5912_b/test.un_nb_5912).fillna(0).astype(np.float16)
test['un_nb_eat_w_ratio']=(test.un_nb_eat_w/test.un_nb_eat).fillna(0).astype(np.float16)
test['un_nb_eat_b_ratio']=(test.un_nb_eat_b/test.un_nb_eat).fillna(0).astype(np.float16)
test['un_nb_fl_w_ratio']=(test.un_nb_fl_w/test.un_nb_fl).fillna(0).astype(np.float16)
test['un_nb_fl_b_ratio']=(test.un_nb_fl_b/test.un_nb_fl).fillna(0).astype(np.float16)
test['un_nb_outh_w_ratio']=(test.un_nb_outh_w/test.un_nb_outh).fillna(0).astype(np.float16)
test['un_nb_outh_b_ratio']=(test.un_nb_outh_b/test.un_nb_outh).fillna(0).astype(np.float16)

In [109]:
train.drop(train[train.fr==0].index, axis=0,inplace=True)
test.drop(test[test.fr==0].index, axis=0,inplace=True)

# Признаки про соседей, которые считаются отдельно для трейна и теста.

In [117]:
#признаки которые надо считать отдельно. количество соседей мест работы из train
knn_work=train[['customer_id', 'work_add_lat' ,'work_add_lon']].dropna().drop_duplicates()[['work_add_lat' ,'work_add_lon']].values.tolist()
neibh = NearestNeighbors(radius=0.02)
neibh.fit(knn_work)
idx=neibh.radius_neighbors(train[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
train['nb_work']=[len(x) for x in idx]-train.is_work.values

idx_val=neibh.radius_neighbors(test[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
test['nb_work']=[len(x) for x in idx_val]

In [118]:
#признаки которые надо считать отдельно. количество соседей домашних адресов из train

knn_home=train[['customer_id', 'home_add_lat' ,'home_add_lon']].dropna().drop_duplicates()[['home_add_lat' ,'home_add_lon']].values.tolist()
neibh = NearestNeighbors(radius=0.02)
neibh.fit(knn_home)
idx=neibh.radius_neighbors(train[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
train['nb_home']=[len(x) for x in idx]-train.is_home.values

idx_val=neibh.radius_neighbors(test[['b_lat', 'b_lon']].values,radius=0.02,return_distance=False)
test['nb_home']=[len(x) for x in idx_val]

In [119]:
train['nb_work1']=train['nb_work']
test['nb_work1']=test['nb_work']

In [ ]:
# Признаки про терминалы, которые связаны с тц и работой

In [130]:
bs=pd.read_csv('bs.csv')
r=0.005
bs_c=bs[['lat','lon']].values.tolist()
neibh = NearestNeighbors(radius=r)
neibh.fit(bs_c)
idx= neibh.radius_neighbors(train[train.is_atm==1][['b_lat', 'b_lon']].values,radius=r,return_distance=False)
train.loc[train.is_atm==1,'bs_c']=[len(x) for x in idx]
train['bs_c']=train['bs_c'].fillna(0).astype(np.int16)

idx= neibh.radius_neighbors(test[test.is_atm==1][['b_lat', 'b_lon']].values,radius=r,return_distance=False)
test.loc[test.is_atm==1,'bs_c']=[len(x) for x in idx]
test['bs_c']=test['bs_c'].fillna(0).astype(np.int16)

In [139]:
#Перейдем к уникальным точкам:

train=train[['amount_mean', 'city', 'customer_id', 'home_add_lat',
       'home_add_lon', 'work_add_lat', 'work_add_lon', 'mcc', 'terminal_id',
       'is_atm', 'b_lat', 'b_lon', 'is_home', 'is_work', 'mcc5912', 'shops',
       'eat', 'fl', 'outh', 'fr', 'dist_cm', 'dist_cm_med', 'n_pnts',
       'fr_ratio', 'fr_w', 'fr_b', 'un_nb', 'un_nb_shops', 'un_nb_atm',
       'un_nb_5912', 'un_nb_eat', 'un_nb_fl', 'un_nb_outh', 'un_nb_shops_w',
       'un_nb_shops_b', 'un_nb_atm_w', 'un_nb_atm_b', 'un_nb_5912_w',
       'un_nb_5912_b', 'un_nb_eat_w', 'un_nb_eat_b', 'un_nb_fl_w',
       'un_nb_fl_b', 'un_nb_outh_w', 'un_nb_outh_b', 'un_nb_shops_ratio',
       'un_nb_5912_ratio', 'un_nb_eat_ratio', 'un_nb_atm_ratio',
       'un_nb_fl_ratio', 'un_nb_atm_w_ratio', 'un_nb_atm_b_ratio',
       'un_nb_shops_w_ratio', 'un_nb_shops_b_ratio', 'un_nb_5912_w_ratio',
       'un_nb_5912_b_ratio', 'un_nb_eat_w_ratio', 'un_nb_eat_b_ratio',
       'un_nb_fl_w_ratio', 'un_nb_fl_b_ratio', 'un_nb_outh_w_ratio',
       'un_nb_outh_b_ratio','nb_work', 'nb_home', 'nb_work1', 'bs_c']].drop_duplicates()

test=test[['amount_mean', 'day_off', 'city', 'customer_id', 'mcc', 'terminal_id',
       'is_atm', 'b_lat', 'b_lon', 'mcc5912', 'shops', 'eat', 'fl', 'outh',
       'fr', 'dist_cm', 'dist_cm_med', 'n_pnts', 'fr_ratio', 'fr_w', 'fr_b',
       'un_nb', 'un_nb_shops', 'un_nb_atm', 'un_nb_5912', 'un_nb_eat',
       'un_nb_fl', 'un_nb_outh', 'un_nb_shops_w', 'un_nb_shops_b',
       'un_nb_atm_w', 'un_nb_atm_b', 'un_nb_5912_w', 'un_nb_5912_b',
       'un_nb_eat_w', 'un_nb_eat_b', 'un_nb_fl_w', 'un_nb_fl_b',
       'un_nb_outh_w', 'un_nb_outh_b', 'un_nb_shops_ratio', 'un_nb_5912_ratio',
       'un_nb_eat_ratio', 'un_nb_atm_ratio', 'un_nb_fl_ratio',
       'un_nb_atm_w_ratio', 'un_nb_atm_b_ratio', 'un_nb_shops_w_ratio',
       'un_nb_shops_b_ratio', 'un_nb_5912_w_ratio', 'un_nb_5912_b_ratio',
       'un_nb_eat_w_ratio', 'un_nb_eat_b_ratio', 'un_nb_fl_w_ratio',
       'un_nb_fl_b_ratio', 'un_nb_outh_w_ratio', 'un_nb_outh_b_ratio','nb_work',
           'nb_home', 'nb_work1', 'bs_c']].drop_duplicates()



# Валидация

In [140]:
# признак для валидации: 0 если известна целевая переменная для дома и работы, 1 если нет координат места работы
# 2 если нет координат дома
train['for_val']=train.groupby('customer_id')['work_add_lat'].transform(lambda x: x.isnull()).to_frame().merge(
    train.groupby('customer_id')['home_add_lat'].transform(lambda x: x.isnull()).to_frame(),
    left_index=True, right_index=True, how='inner').apply(lambda x: x[0]+x[1]*2, axis=1)
#Добавим признак возможно ли определить точку
train['ph'] =train.groupby('customer_id')['is_home'].transform(lambda x: 1 if np.sum(x)>0 else 0)
train['pw'] =train.groupby('customer_id')['is_work'].transform(lambda x: 1 if np.sum(x)>0 else 0)
with open('tr_val.pkl', 'wb') as pkl:
    pickle.dump(train, pkl, protocol=2)

In [238]:
colsw=['amount_mean','terminal_id', 'is_atm', 'mcc5912', 'shops', 'eat', 'fl', 'outh','fr'
      ,'fr_w', 'fr_b','bs_c','nb_work', 'nb_work1','un_nb_eat_b_ratio','un_nb_fl']

colsh=['amount_mean','terminal_id','is_atm', 'mcc5912', 'shops', 'eat', 'fl', 'outh','fr'
      ,'fr_w', 'fr_b','nb_work', 'nb_home',  'un_nb_eat',
       'un_nb_fl', 'un_nb_outh','un_nb_shops','un_nb_shops_w','un_nb_atm','un_nb_5912', 'un_nb_fl_w']


In [236]:
#work
cv_work=[]
val_work=train[(train.for_val!=1)&(train.pw==1)]
gkf = GroupKFold(n_splits=4)
for tr_idx, tst_idx in gkf.split(val_work, val_work['is_work'], groups=val_work['customer_id']):
    tr=val_work.iloc[tr_idx,:]
    ts=val_work.iloc[tst_idx,:]
    xgb1=XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=300,n_jobs=-1,min_child_weight=1,subsample=0.9)
    xgb1.fit(tr[colsw], tr['is_work'])
    y1=xgb1.predict_proba(ts[colsw])[:,1]
    ts['y1']=y1
    a=ts.groupby('customer_id').apply(lambda x: x.is_work[np.argmax(x.y1)])
    cv_work.append(np.sum(a)/len(a))

print (cv_work)
print (np.mean(cv_work))

[0.5154894671623296, 0.5254027261462205, 0.4795539033457249, 0.5452292441140025]
0.516418835192


In [237]:
#home
cv_home=[]
val_home=train[(train.for_val!=2)&(train.ph==1)]
gkf = GroupKFold(n_splits=4)
for tr_idx, tst_idx in gkf.split(val_home, val_home['is_home'], groups=val_home['customer_id']):
    tr=val_home.iloc[tr_idx,:]
    ts=val_home.iloc[tst_idx,:]
    xgb2=XGBClassifier()
    xgb2.fit(tr[colsh], tr['is_home'])
    y2=xgb2.predict_proba(ts[colsh])[:,1]
    ts['y2']=y2
    a=ts.groupby('customer_id').apply(lambda x: x.is_home[np.argmax(x.y2)])
    cv_home.append(np.sum(a)/len(a))
print (cv_home)
print (np.mean(cv_home))

[0.6114072494669509, 0.6098081023454158, 0.6316631130063965, 0.619072988811934]
0.617987863408


# Считаем результат



In [276]:
%%time
with open('le_c1.pkl', 'rb') as f: le_c = pickle.load(f)

tr=train[train.pw==1]
vl=test
xgb1=XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=300,n_jobs=-1,min_child_weight=1,subsample=0.9)
xgb1.fit(tr[colsw], tr['is_work'])
y1=xgb1.predict_proba(vl[colsw])[:,1]

tr=train[train.ph==1]
vl=test
xgb2=XGBClassifier()
xgb2.fit(tr[colsh], tr['is_home'])
y2=xgb2.predict_proba(vl[colsh])[:,1]


ans=test[['customer_id','b_lat','b_lon']]
ans['y_w']=y1
ans['y_h']=y2
h=ans.groupby('customer_id').apply(lambda x: x[['b_lat','b_lon']].loc[np.argmax(x.y_h),:])
w=ans.groupby('customer_id').apply(lambda x: x[['b_lat','b_lon']].loc[np.argmax(x.y_w),:])
df=pd.DataFrame(w.join(h, lsuffix='_w', rsuffix='_h').values, index= le_c.inverse_transform(h.index.astype(int)),
             columns=['_WORK_LAT_','_WORK_LON_','_HOME_LAT_','_HOME_LON_'])
df.to_csv('subm_1.csv', index_label='_ID_')



CPU times: user 2min 33s, sys: 256 ms, total: 2min 34s
Wall time: 2min 34s


In [ ]:
git remote add upstream https://github.com/applied-data-science/Data_Mining_in_Action_2018_Spring
